In [1]:
import csv
import json

# Load the data from the CSV file
import pandas as pd
df_csv = pd.read_csv('/Users/chrishu/Desktop/MLS/tracking_data.csv')


# Load the data from the JSON file
with open('/Users/chrishu/Desktop/MLS/metadata.json', 'r') as f:
    data_json = json.load(f)

home_players = data_json['homePlayers']
df_home = pd.DataFrame(home_players)
data = pd.merge(df_csv, df_home, left_on='player_number', right_on='number')

In [2]:
data['period_id'].unique()

array([1, 2])

In [3]:
# Keep only home players
data = data[data['team_id'] == 6900]


In [4]:
# Get left or right back
data_filter = data[(data['position'] == "LB") | (data['position'] == "RB")]
data_filter["player_id"].unique()

array([246099, 443002])

In [5]:
data_filter["name"].unique()

array(['Ruan', 'Joao Moutinho'], dtype=object)

In [6]:
import pandas as pd
import numpy as np

# offset the position reversing
def adjust_positions_for_period(period_id, x):
    if period_id == 2:
        return x
    else:
        return -x

def count_overlaps(fullback_data, all_players_data):
    
    # Get frames where the fullback's team is in possession
    in_possession = all_players_data[all_players_data['last_touch_team_id'] == fullback_data.iloc[0]['team_id']]

    # Adjust positions based on period_id
    fullback_data['x_adjusted'] = fullback_data.apply(lambda row: adjust_positions_for_period(row['period_id'], row['x']), axis=1)
    all_players_data['x_adjusted'] = all_players_data.apply(lambda row: adjust_positions_for_period(row['period_id'], row['x']), axis=1)
    
    # Get fullback and teammates positions when the team is in possession
    fullback_positions = fullback_data[fullback_data['frame_idx'].isin(in_possession['frame_idx'])]
    teammate_positions = in_possession[in_possession['player_id'] != fullback_data.iloc[0]['player_id']]

    # Merge fullback and teammate positions
    merged_positions = fullback_positions.merge(teammate_positions, on='frame_idx', suffixes=('_fb', '_tm'))
    merged_positions.rename(columns={'x_adjusted_x': 'x_adjusted_fb', 'x_adjusted_y': 'x_adjusted_tm'}, inplace=True)

    # Find overlapping positions
    same_side = np.sign(merged_positions['y_fb'].values) == np.sign(merged_positions['y_tm'].values)
    ahead_of_teammate = merged_positions['x_adjusted_fb'].values > merged_positions['x_adjusted_tm'].values
    ahead_of_ball = merged_positions['x_adjusted_fb'].values > merged_positions['ball_x_fb'].values

    # Count overlapping positions
    overlaps = np.sum(same_side & ahead_of_teammate & ahead_of_ball)

    return overlaps


In [8]:
def main():
    fullback_ids = [246099, 443002]
    fullback_overlaps = {}

    fullback_data = data[data['player_id'].isin(fullback_ids)]

    for player_id in fullback_ids:
        player_data = fullback_data[fullback_data['player_id'] == player_id]
        player_data = player_data.sort_values(by=['period_id', 'frame_idx'])
        fullback_overlaps[player_id] = count_overlaps(player_data, data)
        
    print(fullback_overlaps)


if __name__ == "__main__":
    main()

{246099: 88701, 443002: 52531}
